In [1]:
import torch
import torch.nn as nn

from transformers import AutoTokenizer, AutoConfig, AutoModel

In [2]:
from dataset.dataset import T5Dataset
from dataset.preprocessor.preprocessors import T5BasicPreprocessor
from dataset.augmentation.augmentations import SimpleRandomUNK

In [3]:
MODEL_NAME = "KETI-AIR/ke-t5-small"
config = AutoConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

dataset = T5Dataset("/opt/ml/dataset/")
preprocessor = T5BasicPreprocessor()

dataset.set_tokenizer(tokenizer)
dataset.set_preprocessor(preprocessor)
dataset.preprocess()

Old: 〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.
New: klue_re text: 〈Something〉는 #조지 해리슨#이 쓰고 *비틀즈*가 1969년 앨범 《Abbey Road》에 담은 노래다.
Old: 호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.
New: klue_re text: 호남이 기반인 바른미래당·#대안신당#·*민주평화당*이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.
Old: K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.
New: klue_re text: K리그2에서 성적 1위를 달리고 있는 *광주FC*는 지난 26일 #한국프로축구연맹#으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.
Old: 균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
New: klue_re text: 균일가 생활용품점 (주)*아성다이소*(대표 #박정부#)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
Old: 1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8번으로 배정되었다.
New: klue_re text: #1967#년 프로 야구 드래프트 1순위로 *요미우리 자이언츠*에게 입단하면서 등번호는 8번으로 배정되었다.


In [4]:
dataset.data.head()

,id,sentence,sentence_length,subject_entity_word,subject_entity_start_idx,subject_entity_end_idx,subject_entity_type,object_entity_word,object_entity_start_idx,object_entity_end_idx,object_entity_type,concat_entity,label,source,t5_inputs,t5_sbj_s_idx,t5_sbj_e_idx,t5_obj_s_idx,t5_obj_e_idx
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,59,비틀즈,24,26,ORG,조지 해리슨,13,18,PER,비틀즈[SEP]조지 해리슨,0,wikipedia,klue_re text: 〈Something〉는 #조지 해리슨#이 쓰고 *비틀즈*가...,40,45,27,35
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,54,민주평화당,19,23,ORG,대안신당,14,17,ORG,민주평화당[SEP]대안신당,0,wikitree,klue_re text: 호남이 기반인 바른미래당·#대안신당#·*민주평화당*이 우여...,35,42,28,34
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,99,광주FC,21,24,ORG,한국프로축구연맹,34,41,ORG,광주FC[SEP]한국프로축구연맹,20,wikitree,klue_re text: K리그2에서 성적 1위를 달리고 있는 *광주FC*는 지난 ...,35,41,50,60
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,78,아성다이소,13,17,ORG,박정부,22,24,PER,아성다이소[SEP]박정부,1,wikitree,klue_re text: 균일가 생활용품점 (주)*아성다이소*(대표 #박정부#)는 ...,27,34,38,43
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,56,요미우리 자이언츠,22,30,ORG,1967,0,3,DAT,요미우리 자이언츠[SEP]1967,0,wikipedia,klue_re text: #1967#년 프로 야구 드래프트 1순위로 *요미우리 자이...,38,49,14,20


In [5]:
idx = 0
sentence = dataset.data['t5_inputs'].iloc[0]
print(sentence)

klue_re text: 〈Something〉는 #조지 해리슨#이 쓰고 *비틀즈*가 1969년 앨범 《Abbey Road》에 담은 노래다.


In [6]:
sbj_s_idx = dataset.data['t5_sbj_s_idx'].iloc[0]
sbj_e_idx = dataset.data['t5_sbj_e_idx'].iloc[0]
print(sbj_s_idx, sbj_e_idx)

40 45


In [7]:
sentence[:sbj_s_idx]

'klue_re text: 〈Something〉는 #조지 해리슨#이 쓰고 '

In [10]:
obj_s_idx = dataset.data['t5_obj_s_idx'].iloc[0]
obj_e_idx = dataset.data['t5_obj_e_idx'].iloc[0]
print(sbj_s_idx, sbj_e_idx)

27 35


In [11]:
sentence[:obj_s_idx]

'klue_re text: 〈Something〉는 '

In [15]:
import pandas as pd

In [16]:
train_df = pd.read_csv("/opt/ml/dataset/aeda_bal500_dataset/train/train.csv")
valid_df = pd.read_csv("/opt/ml/dataset/aeda_bal500_dataset/train/valid.csv")

In [17]:
train_df.head()

,id,sentence,subject_entity,object_entity,label,source
0,15584,최성은은 18일 개봉한 '시동'에서 빨간 머리에 까만 선글라스를 끼고 심상치 않은 ...,"{'word': '최성은', 'start_idx': 0, 'end_idx': 2, ...","{'word': ""시동'"", 'start_idx': 14, 'end_idx': 16...",no_relation,wikitree
1,21533,국내 1위 종합숙박·액티비티 예약 서비스 여기어때를 운영하는 위드이노베이션(대표 황...,"{'word': '한국문화관광연구원', 'start_idx': 82, 'end_id...","{'word': '문화체육관광부', 'start_idx': 65, 'end_idx'...",no_relation,wikitree
2,13220,2009년 시즌을 앞두고 SK 와이번스에서 FA를 선언한 외야수 이진영을 LG 트윈...,"{'word': 'SK 와이번스', 'start_idx': 14, 'end_idx'...","{'word': '2009년', 'start_idx': 0, 'end_idx': 4...",no_relation,wikipedia
3,31878,"반면 중국 리그에서 활약 중인 김신욱 선수가 이름을 올렸으며, 울산 현대 미드필더 ...","{'word': '김신욱', 'start_idx': 17, 'end_idx': 19...","{'word': '울산 현대', 'start_idx': 35, 'end_idx': ...",no_relation,wikitree
4,20544,샤론 스톤은 1980년 우디 앨런의 영화 ‘스타더스트 메모리스’로 데뷔해 1990년...,"{'word': '샤론 스톤', 'start_idx': 0, 'end_idx': 4...","{'word': '1980년', 'start_idx': 7, 'end_idx': 1...",no_relation,wikitree


In [22]:
len(train_df)

29170

In [18]:
valid_df.head()

,id,sentence,subject_entity,object_entity,label,source
0,8226,본관은 이천(利川)이고 서울 출생이며 충청남도 서천과 경상북도 달성과 경기도 이천과...,"{'word': 'MBC', 'start_idx': 104, 'end_idx': 1...","{'word': '아리랑', 'start_idx': 190, 'end_idx': 1...",no_relation,wikipedia
1,24342,"당시 LG는 김태원, 김용수, 이상훈 등, 1990년대를 주름잡았던 투수들이 대거 ...","{'word': '이상훈', 'start_idx': 17, 'end_idx': 19...","{'word': '1990년', 'start_idx': 24, 'end_idx': ...",no_relation,wikipedia
2,23725,구독자 114만명을 자랑하는 유튜브 방송 ‘진용진’이 ‘아프리카 큰손들은 뭐 하는 ...,"{'word': '진용진', 'start_idx': 24, 'end_idx': 26...","{'word': '유튜브', 'start_idx': 16, 'end_idx': 18...",no_relation,wikitree
3,18257,이에 3개시와 진해여객이 수용했으나 마창 시내버스 협의회는 난색을 표하다가 결국 8...,"{'word': '진해여객', 'start_idx': 8, 'end_idx': 11...","{'word': '마창 시내버스 협의회', 'start_idx': 20, 'end_...",no_relation,wikipedia
4,4075,펠릭스 4세는 요한 1세가 선종한 지 2개월 후에 교황으로 선출되었다.,"{'word': '펠릭스 4세', 'start_idx': 0, 'end_idx': ...","{'word': '요한 1세', 'start_idx': 8, 'end_idx': 1...",no_relation,wikipedia


In [21]:
len(valid_df)

7329

In [28]:
sum(train_df['id'].isin(valid_df['id']))

0